# Análise dos preços de apartamentos em Salvador

A análise será feita com os dados extraídos na OLX em 04/02/2023.

### Extração dos dados no site da OLX

In [ ]:
import time
#from tqdm.notebook import tqdm

for page_olx in tqdm(range(1,10)):
    time.sleep(2)
    print(page_olx)

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
    'accept-language':'pt-BR,pt;q=0.6',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding':'gzip, deflate'
}


### Funções para web scrape as paginas de anúncios:

In [2]:
#Função para pegar os dados de cada anúncio:
def scrape_olx_item(item):
    #Título do anuncio:
    titulo = item.select("a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-gpHHfC.kJrkKt > h2")
    titulo_anuncio = titulo[0].text

    #Tipo de imóvel:
    tp_imovel_item = item.select("h3")
    tp_imovel_item = [x for x in tp_imovel_item if "venda" in x.text]
    tp_imovel = tp_imovel_item[0].text

    #Informações que não existem em todos os anuncios
    quarto_info, area_info, garagem_info, banheiro_info, iptu_info, condominio_info = '','','','','',''

    #Informações de banheiro, area, quartos e garagem:
    for i in range(1,8):
        try:
            atributo = item.select(f"ul > li:nth-child({i}) > span")[0].attrs['aria-label']
            if 'QUARTO' in atributo.upper():
                quarto_info = atributo
            elif 'METRO' in atributo.upper():
                area_info = atributo
            elif 'GARAGEM' in atributo.upper():
                garagem_info = atributo
            elif 'BANHEIRO' in atributo.upper():
                banheiro_info = atributo 
        except:
            pass

    #Dados de localização:
    localizacao_item = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-kQsIoO.kbWREw > div > div.sc-hjRWVT.lgwjMd > span:nth-child(2)')[0]
    cidade, bairro = [x.strip() for x in localizacao_item.text.split(',')]

    #Valores:
    preco_item = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-cpmKsF.ecSOri > span.main-price')
    preco_imovel = preco_item[0].text

    #Condomínio e IPTU:
    for i in range(1,4):
        try:
            valor_atributo = item.select(f'a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-cpmKsF.ecSOri > span:nth-child({i})')[0].text
            if 'IPTU' in valor_atributo.upper():
                iptu_info = valor_atributo
            elif 'CONDOM' in valor_atributo.upper():
                condominio_info = valor_atributo
        except:
            pass

    #Link do anúncio:
    link_anuncio = item.select('a')[0].attrs['href']
    id_anuncio = link_anuncio.split('-')[-1]


    #Data do anúncio:
    data_anuncio_info = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-kQsIoO.kbWREw > div > div.sc-hjRWVT.lgwjMd > span:nth-child(4)')[0].text

    #Talvez pegar o CEP dentro de cada pagina depois
    return titulo_anuncio, tp_imovel, quarto_info, area_info, garagem_info, banheiro_info, cidade, bairro, preco_imovel, iptu_info, condominio_info, link_anuncio, data_anuncio_info, id_anuncio


#Função para converter quantitativos para string:
def string_to_float_olx(string):
    try:
        return float(string.split()[0])
    except (ValueError, IndexError) as e:
            try:
                return float(string.split()[-1])
            except (ValueError, IndexError) as e:
                return np.nan

#Função para converter a data do anúncio:
def olx_date_to_datetime(date_string):
    if 'HOJE' in date_string.upper():
        date_anuncio = dt.datetime.now()
        return dt.datetime(date_anuncio.year, date_anuncio.month, date_anuncio.day, int(date_string.split(', ')[1].split(':')[0]), int(date_string.split(', ')[1].split(':')[1]))
    elif 'ONTEM' in date_string.upper():
        date_anuncio = dt.datetime.now() + dt.timedelta(days=-1)
        return dt.datetime(date_anuncio.year, date_anuncio.month, date_anuncio.day, int(date_string.split(', ')[1].split(':')[0]), int(date_string.split(', ')[1].split(':')[1])) 
    try:
        hoje = dt.datetime.now()
        day_month, hour_min = date_string.split(', ')
        month_map = {"jan": 1,"fev": 2,"mar": 3,"abr": 4,"mai": 5,"jun": 6,"jul": 7,"ago": 8,"set": 9,"out": 10,"nov": 11,"dez": 12}
        mes = month_map[day_month.split()[-1].lower()]
        dia = int(day_month.split()[0])
        hora, minuto = map(int, hour_min.split(":"))
        candidate_date = dt.datetime(hoje.year, mes, dia, hora, minuto)

        if candidate_date > hoje:
            ano = hoje.year - 1
        else:
            ano = hoje.year
        return dt.datetime(ano, mes, dia, hora, minuto)
    except (ValueError, KeyError):
        return None


In [3]:
for page_olx in tqdm(range(1,101)):
    time.sleep(2)
    URL = "https://www.olx.com.br/imoveis/venda/estado-ba/grande-salvador/salvador?o={page_olx}"

    response = requests.get(url=URL,headers=headers)
    salvador_olx_data = response.text
    soup = BeautifulSoup(salvador_olx_data, "html.parser")

    page_data_items = soup.select("#ad-list > li")

    for item in page_data_items:
        if not item.attrs['class'][0] == 'sponsored':
            try:
                #Pegar dados de cada item:
                titulo_anuncio, tp_imovel, quarto_info, area_info, garagem_info, banheiro_info, cidade, bairro, preco_imovel, iptu_info, condominio_info, link_anuncio, data_anuncio_info, id_anuncio = scrape_olx_item(item)

                #Converter dados quantitativos para foat:
                qtd_quartos = string_to_float_olx(quarto_info)
                area_m2 = string_to_float_olx(area_info)
                vagas_garagem = string_to_float_olx(garagem_info)
                qtd_banheiros = string_to_float_olx(banheiro_info)
                valor_imovel = string_to_float_olx(preco_imovel.replace('.',''))
                valor_iptu = string_to_float_olx(iptu_info.replace('.',''))
                valor_condominio = string_to_float_olx(condominio_info.replace('.',''))

                #Converter data do anúncio para data:
                data_anuncio_datetime = olx_date_to_datetime(data_anuncio_info).strftime('%Y-%m-%d %H:%M')

                #Criação do Dataframe:
                anuncio_dict = {
                    'ID_ANUNCIO': id_anuncio,
                    'TIPO_IMOVEL': tp_imovel.split()[0],
                    'TITULO': titulo_anuncio,
                    'QTD_QUARTOS': qtd_quartos,
                    'AREA_M2': area_m2,
                    'VAGAS_GARAGEM': vagas_garagem,
                    'QTD_BANHEIROS': qtd_banheiros,
                    'CIDADE': cidade,
                    'BAIRRO': bairro,
                    'VALOR_RS': valor_imovel,
                    'IPTU_RS': valor_iptu,
                    'CONDOMINIO_RS': valor_condominio,
                    'DATA_PUBLICACAO': data_anuncio_datetime,
                    'LINK': link_anuncio,
                }

                df_anuncio_id = pd.DataFrame(data=anuncio_dict.items()).T
                df_anuncio_id.columns = df_anuncio_id.iloc[0]
                df_anuncio_id = df_anuncio_id[1:]

                #Agrupar anuncios em data frame TT:
                try:
                    df_anuncios_tt = pd.concat([df_anuncios_tt, df_anuncio_id])
                except:
                    df_anuncios_tt = df_anuncio_id
            except IndexError:
                pass

  0%|          | 0/100 [00:03<?, ?it/s]


IndexError: list index out of range

In [4]:
df_anuncios_tt.reset_index(inplace=True)
df_anuncios_tt.drop_duplicates(inplace=True)
df_anuncios_tt = df_anuncios_tt[list(anuncio_dict.keys())]
df_anuncios_tt.head()

,ID_ANUNCIO,TIPO_IMOVEL,TITULO,QTD_QUARTOS,AREA_M2,VAGAS_GARAGEM,QTD_BANHEIROS,CIDADE,BAIRRO,VALOR_RS,IPTU_RS,CONDOMINIO_RS,DATA_PUBLICACAO,LINK
0,1028143532,Apartamento,Apartamento para venda possui 143 metros quadr...,3.0,143.0,3.0,4.0,Salvador,Horto Bela Vista,1190000.0,2500.0,900.0,2023-02-05 14:48,https://ba.olx.com.br/grande-salvador/imoveis/...
1,1005733008,Casa,Casa para venda possui 340 metros quadrados co...,5.0,340.0,4.0,3.0,Salvador,Stella Maris,1200000.0,900.0,NaN,2023-02-05 14:48,https://ba.olx.com.br/grande-salvador/imoveis/...
2,1095718713,Apartamento,"Apartamento à venda, 2 quartos, 1 vaga, São Cr...",2.0,46.0,1.0,1.0,Salvador,São Cristóvão,155000.0,40.0,NaN,2023-02-05 14:48,https://ba.olx.com.br/grande-salvador/imoveis/...
3,1034560632,Apartamento,"Apartamento com 3 quartos, 70m2, à venda em Sa...",3.0,70.0,1.0,3.0,Salvador,Costa Azul,449000.0,850.0,680.0,2023-02-05 14:48,https://ba.olx.com.br/grande-salvador/imoveis/...
4,1095457928,Apartamento,Apartamento à venda Port. Fechada tem 88m2 com...,3.0,88.0,1.0,1.0,Salvador,Patamares,820000.0,1600.0,NaN,2023-02-05 14:48,https://ba.olx.com.br/grande-salvador/imoveis/...


In [5]:
df_anuncios_tt.describe()

,ID_ANUNCIO,TIPO_IMOVEL,TITULO,QTD_QUARTOS,AREA_M2,VAGAS_GARAGEM,QTD_BANHEIROS,CIDADE,BAIRRO,VALOR_RS,IPTU_RS,CONDOMINIO_RS,DATA_PUBLICACAO,LINK
count,33,33,33,33.0,33.0,30.0,33.0,33,33,33.0,18.0,20.0,33,33
unique,33,2,33,5.0,29.0,5.0,5.0,1,24,30.0,18.0,18.0,9,33
top,1028143532,Apartamento,Apartamento para venda possui 143 metros quadr...,2.0,80.0,1.0,1.0,Salvador,Piatã,550000.0,2500.0,900.0,2023-02-05 14:39,https://ba.olx.com.br/grande-salvador/imoveis/...
freq,1,20,1,10.0,3.0,15.0,13.0,33,3,2.0,1.0,2.0,11,1


### Rascunhos Web Scraping:

In [176]:
#Lista de itens na página
page_data_items = soup.select("#ad-list > li")
item = page_data_items[10]

#Título do anuncio:
titulo = item.select("a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-gpHHfC.kJrkKt > h2")
titulo_anuncio = titulo[0].text

#Tipo de imóvel:
tp_imovel_item = item.select("h3")
tp_imovel_item = [x for x in tp_imovel_item if "venda" in x.text]
tp_imovel = tp_imovel_item[0].text

#Informações que não existem em todos os anuncios
quarto_info, area_info, garagem_info, banheiro_info, iptu_info, condominio_info = '','','','','',''

#Informações de banheiro, area, quartos e garagem:
for i in range(1,8):
    try:
        atributo = item.select(f"ul > li:nth-child({i}) > span")[0].attrs['aria-label']
        if 'QUARTO' in atributo.upper():
            quarto_info = atributo
        elif 'METRO' in atributo.upper():
            area_info = atributo
        elif 'GARAGEM' in atributo.upper():
            garagem_info = atributo
        elif 'BANHEIRO' in atributo.upper():
            banheiro_info = atributo 
    except:
        pass

#Dados de localização:
localizacao_item = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-kQsIoO.kbWREw > div > div.sc-hjRWVT.lgwjMd > span:nth-child(2)')[0]
cidade, bairro = [x.strip() for x in localizacao_item.text.split(',')]

#Valores:
preco_item = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-cpmKsF.ecSOri > span.main-price')
preco_imovel = preco_item[0].text

#Condomínio e IPTU:
for i in range(1,4):
    try:
        valor_atributo = item.select(f'a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-cpmKsF.ecSOri > span:nth-child({i})')[0].text
        if 'IPTU' in valor_atributo.upper():
            iptu_info = valor_atributo
        elif 'CONDOM' in valor_atributo.upper():
            condominio_info = valor_atributo
    except:
        pass

#Link do anúncio:
link_anuncio = item.select('a')[0].attrs['href']
id_anuncio = link_anuncio.split('-')[-1]


#Data do anúncio:
data_anuncio_info = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-kQsIoO.kbWREw > div > div.sc-hjRWVT.lgwjMd > span:nth-child(4)')[0].text

#Talvez pegar o CEP dentro de cada pagina depois


In [177]:
print(f'{titulo_anuncio}\n{tp_imovel}\n{quarto_info}\n{area_info}\n{garagem_info}\n{banheiro_info}\n{cidade}\n{bairro}\n{preco_imovel}\n{iptu_info}\n{condominio_info}\n{link_anuncio}\n{data_anuncio_info}')


Apartamento para venda possui 220 metros quadrados com 4 quartos em Costa Azul - Salvador
Apartamento À venda
4 quartos
220 metros quadrados
1 vaga de garagem
4 banheiros
Salvador
Costa Azul
R$ 695.000


https://ba.olx.com.br/grande-salvador/imoveis/apartamento-para-venda-possui-220-metros-quadrados-com-4-quartos-em-costa-azul-salvador-956908426
Hoje, 09:53


In [178]:
#Convertendo os dados quantitativos para string:
def string_to_float_olx(string):
    try:
        return float(string.split()[0])
    except (ValueError, IndexError) as e:
            try:
                return float(string.split()[-1])
            except (ValueError, IndexError) as e:
                return np.nan
            
qtd_quartos = string_to_float_olx(quarto_info)
area_m2 = string_to_float_olx(area_info)
vagas_garagem = string_to_float_olx(garagem_info)
qtd_banheiros = string_to_float_olx(banheiro_info)
valor_imovel = string_to_float_olx(preco_imovel.replace('.',''))
valor_iptu = string_to_float_olx(iptu_info.replace('.',''))
valor_condominio = string_to_float_olx(condominio_info.replace('.',''))

In [179]:
# Função para converter as datas dos anúncios:
def olx_date_to_datetime(date_string):
    if 'HOJE' in date_string.upper():
        date_anuncio = dt.datetime.now()
        return dt.datetime(date_anuncio.year, date_anuncio.month, date_anuncio.day, int(date_string.split(', ')[1].split(':')[0]), int(date_string.split(', ')[1].split(':')[1]))
    elif 'ONTEM' in date_string.upper():
        date_anuncio = dt.datetime.now() + dt.timedelta(days=-1)
        return dt.datetime(date_anuncio.year, date_anuncio.month, date_anuncio.day, int(date_string.split(', ')[1].split(':')[0]), int(date_string.split(', ')[1].split(':')[1])) 
    try:
        hoje = dt.datetime.now()
        day_month, hour_min = date_string.split(', ')
        month_map = {"jan": 1,"fev": 2,"mar": 3,"abr": 4,"mai": 5,"jun": 6,"jul": 7,"ago": 8,"set": 9,"out": 10,"nov": 11,"dez": 12}
        mes = month_map[day_month.split()[-1].lower()]
        dia = int(day_month.split()[0])
        hora, minuto = map(int, hour_min.split(":"))
        candidate_date = dt.datetime(hoje.year, mes, dia, hora, minuto)

        if candidate_date > hoje:
            ano = hoje.year - 1
        else:
            ano = hoje.year
        return dt.datetime(ano, mes, dia, hora, minuto)
    except (ValueError, KeyError):
        return None

#Teste função
# data1 = 'Hoje, 08:12'
# data2 = '3 de fev, 16:18'
# data3 = 'Ontem, 23:58'

# olx_date_to_datetime(data1), olx_date_to_datetime(data2), olx_date_to_datetime(data3)

data_anuncio_datetime = olx_date_to_datetime(data_anuncio_info).strftime('%Y-%m-%d %H:%M')


In [180]:
#Criação do Dataframe:
anuncio_dict = {
    'ID_ANUNCIO': id_anuncio,
    'TIPO_IMOVEL': tp_imovel,
    'TITULO': titulo_anuncio,
    'QTD_QUARTOS': qtd_quartos,
    'AREA_M2': area_m2,
    'VAGAS_GARAGEM': vagas_garagem,
    'QTD_BANHEIROS': qtd_banheiros,
    'CIDADDE': cidade,
    'BAIRRO': bairro,
    'VALOR_RS': valor_imovel,
    'IPTU_RS': valor_iptu,
    'CONDOMINIO_RS': valor_condominio,
    'DATA_PUBLICACAO': data_anuncio_datetime,
    'LINK': link_anuncio,
}

df_anuncio_id = pd.DataFrame(data=anuncio_dict.items()).T
df_anuncio_id.columns = df_anuncio_id.iloc[0]
df_anuncio_id = df_anuncio_id[1:]

#Agrupar anuncios em data frame TT:
try:
    df_anuncios_tt = pd.concat([df_anuncios_tt, df_anuncio_id])
except:
    df_anuncios_tt = df_anuncio_id

In [181]:
df_anuncios_tt


,ID_ANUNCIO,TIPO_IMOVEL,TITULO,QTD_QUARTOS,AREA_M2,VAGAS_GARAGEM,QTD_BANHEIROS,CIDADDE,BAIRRO,VALOR_RS,IPTU_RS,CONDOMINIO_RS,DATA_PUBLICACAO,LINK
1,1113267526,Apartamento À venda,VENDO APARTAMENTO - 59 M² - 1 QUARTOS - CELEBR...,1.0,59.0,1.0,1.0,Salvador,Rio Vermelho,650000.0,101.0,423.0,2023-02-05 08:12,https://ba.olx.com.br/grande-salvador/imoveis/...
1,956908426,Apartamento À venda,Apartamento para venda possui 220 metros quadr...,4.0,220.0,1.0,4.0,Salvador,Costa Azul,695000.0,NaN,NaN,2023-02-05 09:53,https://ba.olx.com.br/grande-salvador/imoveis/...


In [182]:
df_anuncios_tt.LINK.to_list()

['https://ba.olx.com.br/grande-salvador/imoveis/vendo-apartamento-59-m-1-quartos-celebration-garibaldi-rio-vermelhor-salvador-1113267526',
 'https://ba.olx.com.br/grande-salvador/imoveis/apartamento-para-venda-possui-220-metros-quadrados-com-4-quartos-em-costa-azul-salvador-956908426']